## Problema do Pecuarista

In [ ]:
!pip install pulp
import pulp

# Criar o problema de minimização
prob = pulp.LpProblem("Pecuarista", pulp.LpMinimize)

# Criar as variáveis de decisão
x1 = pulp.LpVariable("milho", lowBound=0, cat='Continuous')
x2 = pulp.LpVariable("ração", lowBound=0, cat='Continuous')
x3 = pulp.LpVariable("alfafa", lowBound=0, cat='Continuous')

# Definir a função objetivo
prob += 21*x1 + 18*x2 + 15*x3, "Custo Total de Produção"

# Restrição de produção de carboidratos
prob += 90*x1 + 20*x2 + 40*x3 >= 200, "Produção de Carboidratos"

# Restrição de produção de proteinas
prob += 30*x1 + 80*x2 + 60*x3 >= 180, "Produção de Proteínas"

# Restrição de produção de vitaminas
prob += 10*x1 + 20*x2 + 60*x3 >= 150, "Produção de Vitaminas"

# Resolver o problema
prob.solve()

# Imprimir a solução
print(prob)
print("Status:", pulp.LpStatus[prob.status])
print("Custo Mínimo Diário de Produção = R$", pulp.value(prob.objective))
print(f'x1: {x1.value()}\nx2: {x2.value()} \nx3: {x3.value()}')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 43.7 MB/s eta 0:00:00
Pecuarista:
MINIMIZE
15*alfafa + 21*milho + 18*ração + 0
SUBJECT TO
Produção_de_Carboidratos: 40 alfafa + 90 milho + 20 ração >= 200

Produção_de_Proteínas: 60 alfafa + 30 milho + 80 ração >= 180

Produção_de_Vitaminas: 60 alfafa + 10 milho + 20 ração >= 150

VARIABLES
alfafa Continuous
milho Continuous
ração Continuous

Status: Optimal
Custo Mínimo Diário de Produção = R$ 60.4285701
x1: 1.1428571
x2: 0.0 
x3: 2.4285714


## Tabela Simplex

In [ ]:
import numpy as np
np.set_printoptions(precision=3, suppress=True)
import warnings
warnings.filterwarnings('ignore')
import pandas as pd


def Simplex(T, rotulos=[], base=[]):
    '''
    Função para calcular o Tableau Simplex apresentando a sua evolução ao longo das iterações.
    
    Argumentos de entrada:
        T: numpy array representando o Tableau inicial
    
    Argumentos de Saída
        T: a tabela na última iteração do algoritmo
        
    Programador: Prof. Dr. Rooney Coelho
    '''
    
    print('Tableu Simplex (inicial):')
    if rotulos == [] and base == []:
        print(T)
    else:
        print( pd.DataFrame(T, columns=rotulos, index=base ) )
    
    menor = -1
    it = 0
    while menor < 0:
        # Inicialização dos parâmetros (sobreescrever)
        pivo_linha = -1
        pivo_coluna = -1
        pivo = 0
        menor = T[0,:-1].min()

        if menor >=0:
            print('\nNenhum dos coeficientes da linha z associados com as variáveis não básicas é negativo assim essa tabela é ótima!')
            break
        else:
            it += 1
            print(f'\nIteração: {it}')
            # pega o menor elemento da primeira linha (função objetivo)
            pivo_coluna = T[0,:-1].argmin()

            aux = np.zeros(len(T)-1)
            i = 0
            for a,b in zip(T[1:,-1],T[1:,pivo_coluna]):
                aux[i] = a/b
                i+=1

            val = aux[aux>0].min()
            pivo_linha = np.argwhere(aux==val).item() + 1 # Soma um para a mesma referencia incluindo objetivo
            pivo = T[pivo_linha, pivo_coluna]
            print(f'Linha do pivô: {pivo_linha}, Coluna do pivô: {pivo_coluna}, Elemento pivô: {pivo}\n')

            # Nova linha do pivô = linha do pivô atual / elemento do pivô
            T[pivo_linha] = T[pivo_linha]/pivo 

            # Todas as outras linhas, incluindo z
            # Nova linha=(Linha atual)−(seu coeficiente de coluna do pivô)×(Nova linha do pivô)

            nova_linha_pivo = T[pivo_linha]

            for i in range(len(T)):
                if i != pivo_linha:
                    T[i] -= T[i,pivo_coluna]*nova_linha_pivo

            print('Tableu Simplex:')
            if rotulos == [] and base == []:
                print(T)
            else:
                base[pivo_linha] = rotulos[pivo_coluna]
                print( pd.DataFrame(T, columns=rotulos, index=base ) )
    return T

In [ ]:
T = np.array([
    [1,-5,-4,0,0,0,0,0],
    [0,6,4,1,0,0,0,24],
    [0,1,2,0,1,0,0,6],
    [0,-1,1,0,0,1,0,1],
    [0,0,1,0,0,0,1,2],
], dtype=float)

rotulos = ['z', 'x1', 'x2', 's1', 's2', 's3', 's4', 'RHD' ]
base = ['z', 's1', 's2', 's3', 's4']

Simplex(T, rotulos, base);

Tableu Simplex (inicial):
      z   x1   x2   s1   s2   s3   s4   RHD
z   1.0 -5.0 -4.0  0.0  0.0  0.0  0.0   0.0
s1  0.0  6.0  4.0  1.0  0.0  0.0  0.0  24.0
s2  0.0  1.0  2.0  0.0  1.0  0.0  0.0   6.0
s3  0.0 -1.0  1.0  0.0  0.0  1.0  0.0   1.0
s4  0.0  0.0  1.0  0.0  0.0  0.0  1.0   2.0

Iteração: 1
Linha do pivô: 1, Coluna do pivô: 1, Elemento pivô: 6.0

Tableu Simplex:
      z   x1        x2        s1   s2   s3   s4   RHD
z   1.0  0.0 -0.666667  0.833333  0.0  0.0  0.0  20.0
x1  0.0  1.0  0.666667  0.166667  0.0  0.0  0.0   4.0
s2  0.0  0.0  1.333333 -0.166667  1.0  0.0  0.0   2.0
s3  0.0  0.0  1.666667  0.166667  0.0  1.0  0.0   5.0
s4  0.0  0.0  1.000000  0.000000  0.0  0.0  1.0   2.0

Iteração: 2
Linha do pivô: 2, Coluna do pivô: 2, Elemento pivô: 1.3333333333333335

Tableu Simplex:
      z   x1   x2     s1    s2   s3   s4   RHD
z   1.0  0.0  0.0  0.750  0.50  0.0  0.0  21.0
x1  0.0  1.0  0.0  0.250 -0.50  0.0  0.0   3.0
x2  0.0  0.0  1.0 -0.125  0.75  0.0  0.0   1.5
s3  0.0  0.